# RMSA Experimental Notebook
## Análisis determinista de algoritmos RMSA sobre bancos TOP

Este notebook compara First-Fit, Parcel-Fit y Sliding-Fit en escenarios RMSA estáticos. Ahora se soporta:

- Descubrimiento automático de todas las topologías `TOP_XX_ALIAS` dentro de `Rutas/`.
- Configuración para ejecutar un subconjunto o el banco completo sin editar código.
- Uso del valor real de slots definido en cada topología para construir la capacidad.
- Evaluación "todos-contra-todos" sin demandas aleatorias ni métricas de probabilidad de bloqueo.
- Ordenamiento de rutas por número de saltos antes de asignar espectro para emular una política hop-first.

### Instalación de dependencias

In [ ]:
!pip3 install numpy pandas plotly networkx

### Importaciones necesarias

In [3]:
import json
import os
import re
import sys
import time
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Definir rutas relevantes del proyecto sin asumir carpeta de ejecución
WORKSPACE_DIR = Path(os.getcwd()).resolve()
PROJECT_ROOT = WORKSPACE_DIR if (WORKSPACE_DIR / 'Rutas').exists() else WORKSPACE_DIR.parent
NOTEBOOKS_DIR = PROJECT_ROOT / 'notebooks'

for candidate in (NOTEBOOKS_DIR, PROJECT_ROOT):
    if candidate.exists() and str(candidate) not in sys.path:
        sys.path.insert(0, str(candidate))

RUTAS_DIR = PROJECT_ROOT / 'Rutas'
RESULTS_DIR = PROJECT_ROOT / 'results'


### Importación de funciones del loader

In [4]:
try:
    from src.loader import obtener_enlaces_directos, crear_rutas_usuarios
except ModuleNotFoundError:
    from notebooks.src.loader import obtener_enlaces_directos, crear_rutas_usuarios

try:
    from src.algorithms.first_fit import find_first_fit_and_allocate
    from src.algorithms.parcel_fit import allocate_parcel_fit
    from src.algorithms.sliding_fit import allocate_sliding_fit
except ModuleNotFoundError:
    from notebooks.src.algorithms.first_fit import find_first_fit_and_allocate
    from notebooks.src.algorithms.parcel_fit import allocate_parcel_fit
    from notebooks.src.algorithms.sliding_fit import allocate_sliding_fit

### Configuración del experimento

Usa esta celda para elegir si se ejecuta sobre todas las topologías disponibles en `Rutas/` o sólo sobre un subconjunto (`selected`). Cambia la lista `SELECTED_TOPOLOGIES` para fijar uno o varios códigos `TOP_XX_ALIAS`. También puedes ajustar cuántos slots requiere cada conexión y el tamaño de parcela usado por Parcel-Fit.

In [ ]:
ANALYSIS_MODE = "selected"  # "selected" o "all"
SELECTED_TOPOLOGIES = [
    "TOP_1_ABILENE",
    "TOP_2_ANS",
    "TOP_3_ARNES",
]

DEMAND_SLOTS = 4            # Slots requeridos por cada par origen-destino
PARCEL_SIZE = 2             # Tamaño de parcela para Parcel-Fit

assert ANALYSIS_MODE in {"selected", "all"}, "ANALYSIS_MODE debe ser 'selected' o 'all'"


### Definición de métricas y funciones auxiliares

In [16]:
def fragmentation_metric(capacity):
    """Promedio de segmentos libres por enlace (indicador de fragmentación)."""
    segments = []
    for row in capacity:
        free = (row == 0).astype(int)
        seg = 0
        inblock = False
        for value in free:
            if value == 1 and not inblock:
                seg += 1
                inblock = True
            elif value == 0:
                inblock = False
        segments.append(seg)
    return float(np.mean(segments)) if segments else 0.0


def spectrum_efficiency(capacity):
    """Proporción de ranuras utilizadas respecto al total disponible."""
    if capacity.size == 0:
        return 0.0
    return float(np.sum(capacity)) / capacity.size


TOPO_PATTERN = re.compile(r"^(TOP_(\d+)_.+?)_routes\.json$", re.IGNORECASE)


def discover_topologies(rutas_dir: Path):
    """Localiza parejas (topología, rutas) y retorna metadatos ordenados por índice TOP."""
    rutas_dir = Path(rutas_dir)
    discovered = []
    for routes_file in rutas_dir.glob("TOP_*_routes.json"):
        match = TOPO_PATTERN.match(routes_file.name)
        if not match:
            continue
        base_name = match.group(1)
        topo_file = rutas_dir / f"{base_name}.json"
        if not topo_file.exists():
            continue
        discovered.append({
            "name": base_name,
            "numeric_id": int(match.group(2)),
            "routes_path": routes_file,
            "topology_path": topo_file,
        })
    return sorted(discovered, key=lambda item: item["numeric_id"])


def extract_slot_count(topo_path: Path, default_slots: int = 80) -> int:
    try:
        with open(topo_path, "r", encoding="utf-8") as handler:
            topo_data = json.load(handler)
        slot_values = {
            int(link.get("slots", default_slots))
            for link in topo_data.get("links", [])
            if "slots" in link
        }
        return min(slot_values) if slot_values else default_slots
    except FileNotFoundError:
        return default_slots


def load_topology_payload(meta: dict):
    with open(meta["routes_path"], "r", encoding="utf-8") as handler:
        routes_payload = json.load(handler)
    routes_df = pd.json_normalize(routes_payload["routes"])
    routes_df["paths"] = routes_df["paths"].apply(lambda paths: [paths[0]] if paths else [])
    enlaces = obtener_enlaces_directos(routes_df)
    rutas_usuarios = crear_rutas_usuarios(routes_df, enlaces)
    slots_per_link = extract_slot_count(meta["topology_path"])
    return {
        **meta,
        "alias": routes_payload.get("alias", meta["name"]),
        "routes_df": routes_df,
        "enlaces": enlaces,
        "rutas_usuarios": rutas_usuarios,
        "slots_per_link": slots_per_link,
        "n_connections": len(routes_df),
        "n_enlaces": len(enlaces),
    }


def sort_routes_by_hops(rutas_usuarios: np.ndarray):
    if rutas_usuarios.size == 0:
        return []
    hop_lengths = [int(np.sum(row != -1)) for row in rutas_usuarios]
    return list(np.argsort(hop_lengths))


def build_workload(rutas_usuarios: np.ndarray):
    ordered_indices = sort_routes_by_hops(rutas_usuarios)
    return [rutas_usuarios[idx] for idx in ordered_indices]


def run_allocator(workload, enlaces, slots_per_link, allocator_fn, demand_slots, extra_kwargs=None):
    extra_kwargs = extra_kwargs or {}
    capacity = np.zeros((len(enlaces), slots_per_link), dtype=np.int8)
    blocked = 0
    start_time = time.time()
    for route_row in workload:
        _, ok = allocator_fn(capacity, route_row, demand_slots, **extra_kwargs)
        if not ok:
            blocked += 1
    elapsed = time.time() - start_time
    return {
        "blocked_connections": blocked,
        "served_connections": len(workload) - blocked,
        "utilization": spectrum_efficiency(capacity),
        "fragmentation": fragmentation_metric(capacity),
        "time_s": elapsed,
    }


### Runner de experimentos RMSA

In [17]:
def run_rmsa_experiment(topology_payload, demand_slots=DEMAND_SLOTS, parcel_size=PARCEL_SIZE):
    """Ejecuta los algoritmos RMSA sobre todos los pares origen-destino de la topología."""
    rutas_usuarios = topology_payload["rutas_usuarios"]
    workload = build_workload(rutas_usuarios)
    enlaces = topology_payload["enlaces"]
    slots_per_link = topology_payload["slots_per_link"]

    if len(workload) == 0:
        return {"workload_size": 0}

    results = {
        "first_fit": run_allocator(workload, enlaces, slots_per_link, find_first_fit_and_allocate, demand_slots),
        "sliding_fit": run_allocator(workload, enlaces, slots_per_link, allocate_sliding_fit, demand_slots),
        "parcel_fit": run_allocator(
            workload,
            enlaces,
            slots_per_link,
            allocate_parcel_fit,
            demand_slots,
            extra_kwargs={"parcel_size": parcel_size},
        ),
        "workload_size": len(workload),
    }
    return results


### Descubrimiento y carga de topologías TOP

In [18]:
available_topologies = discover_topologies(RUTAS_DIR)
if not available_topologies:
    raise RuntimeError(f"No se encontraron archivos TOP en {RUTAS_DIR}")

if ANALYSIS_MODE == "all":
    target_topologies = available_topologies
else:
    available_map = {meta["name"]: meta for meta in available_topologies}
    missing = [name for name in SELECTED_TOPOLOGIES if name not in available_map]
    if missing:
        print(f"Advertencia: no se encontraron {missing}. Se ignorarán.")
    target_topologies = [available_map[name] for name in SELECTED_TOPOLOGIES if name in available_map]

if not target_topologies:
    raise RuntimeError("No hay topologías seleccionadas para analizar.")

loaded_topos = {}
for meta in target_topologies:
    payload = load_topology_payload(meta)
    loaded_topos[payload["name"]] = payload
    print(
        f"{payload['name']}: alias={payload['alias']}, rutas={payload['n_connections']}, "
        f"enlaces={payload['n_enlaces']}, slots/link={payload['slots_per_link']}"
    )

print(f"\nTotal de topologías seleccionadas: {len(loaded_topos)}")


TOP_1_ABILENE: alias=ABILENE, rutas=110, enlaces=28, slots/link=320
TOP_2_ANS: alias=ANS, rutas=306, enlaces=50, slots/link=320
TOP_3_ARNES: alias=ARNES, rutas=272, enlaces=40, slots/link=320
TOP_4_ARPANET: alias=ARPANET, rutas=380, enlaces=64, slots/link=320
TOP_5_ATMNET: alias=ATMNET, rutas=420, enlaces=44, slots/link=320
TOP_6_BBNPLANET: alias=BBNPLANET, rutas=702, enlaces=56, slots/link=320
TOP_7_BEYONDTHENETWORK: alias=BEYONDTHENETWORK, rutas=812, enlaces=82, slots/link=320
TOP_8_BICS: alias=BICS, rutas=1056, enlaces=96, slots/link=320
TOP_9_BIZNET: alias=BIZNET, rutas=756, enlaces=64, slots/link=320
TOP_10_BREN: alias=BREN, rutas=90, enlaces=22, slots/link=320
TOP_11_CANARIE: alias=CANARIE, rutas=552, enlaces=66, slots/link=320
TOP_12_CANARIE: alias=CANARIE, rutas=342, enlaces=52, slots/link=320
TOP_13_CERNET: alias=CERNET, rutas=1260, enlaces=102, slots/link=320
TOP_14_CESNET: alias=CESNET, rutas=132, enlaces=34, slots/link=320
TOP_15_CLARANET: alias=CLARANET, rutas=210, enlaces

### Ejecución determinista de los algoritmos RMSA

In [19]:
all_results = {}

for topo_name, topo_data in loaded_topos.items():
    alias = topo_data.get("alias", topo_name)
    print(f"\n=== {topo_name} ({alias}) ===")
    results = run_rmsa_experiment(topo_data)
    all_results[topo_name] = results
    workload_size = results.get("workload_size", len(topo_data["routes_df"]))
    print(f"  Conexiones evaluadas: {workload_size}")
    for alg_name in ("first_fit", "sliding_fit", "parcel_fit"):
        metrics = results.get(alg_name, {})
        if not metrics:
            continue
        print(
            f"  {alg_name}: util={metrics['utilization']:.4f}, "
            f"frag={metrics['fragmentation']:.2f}, blocked={metrics['blocked_connections']}"
        )



=== TOP_1_ABILENE (ABILENE) ===
  Conexiones evaluadas: 110
  first_fit: util=0.1214, frag=3.68, blocked=0
  sliding_fit: util=0.1214, frag=4.39, blocked=0
  parcel_fit: util=0.1214, frag=3.68, blocked=0

=== TOP_2_ANS (ANS) ===
  Conexiones evaluadas: 306
  first_fit: util=0.2285, frag=6.78, blocked=0
  sliding_fit: util=0.2285, frag=7.34, blocked=0
  parcel_fit: util=0.2285, frag=6.78, blocked=0

=== TOP_3_ARNES (ARNES) ===
  Conexiones evaluadas: 306
  first_fit: util=0.2285, frag=6.78, blocked=0
  sliding_fit: util=0.2285, frag=7.34, blocked=0
  parcel_fit: util=0.2285, frag=6.78, blocked=0

=== TOP_3_ARNES (ARNES) ===
  Conexiones evaluadas: 272
  first_fit: util=0.2656, frag=7.33, blocked=0
  sliding_fit: util=0.2656, frag=7.92, blocked=0
  parcel_fit: util=0.2656, frag=7.33, blocked=0

=== TOP_4_ARPANET (ARPANET) ===
  Conexiones evaluadas: 272
  first_fit: util=0.2656, frag=7.33, blocked=0
  sliding_fit: util=0.2656, frag=7.92, blocked=0
  parcel_fit: util=0.2656, frag=7.33, b

### Visualización interactiva con líneas y range slider

In [20]:
ALGORITHM_LABELS = {
    "first_fit": "First-Fit",
    "sliding_fit": "Sliding-Fit",
    "parcel_fit": "Parcel-Fit",
}
LABEL_TO_KEY = {label: key for key, label in ALGORITHM_LABELS.items()}

if not all_results:
    raise RuntimeError("No hay resultados para graficar. Ejecuta la celda de experimentos primero.")

ordered_payloads = sorted(loaded_topos.values(), key=lambda topo: topo["numeric_id"])
plot_rows = []
for payload in ordered_payloads:
    topo_results = all_results.get(payload["name"], {})
    for alg_key, alg_label in ALGORITHM_LABELS.items():
        metrics = topo_results.get(alg_key)
        if not metrics:
            continue
        plot_rows.append({
            "rank": payload["numeric_id"],
            "topology": payload["name"],
            "alias": payload["alias"],
            "algorithm": alg_label,
            "utilization": metrics["utilization"],
            "fragmentation": metrics["fragmentation"],
        })

df_metrics = pd.DataFrame(plot_rows)
if df_metrics.empty:
    raise RuntimeError("No se generaron métricas para graficar (¿rutas vacías?).")

tick_vals = [payload["numeric_id"] for payload in ordered_payloads]
tick_text = [payload["name"] for payload in ordered_payloads]

fig_util = go.Figure()
for alg_label in df_metrics["algorithm"].unique():
    alg_df = df_metrics[df_metrics["algorithm"] == alg_label].sort_values("rank")
    fig_util.add_trace(
        go.Scatter(
            x=alg_df["rank"],
            y=alg_df["utilization"],
            mode="lines+markers",
            name=alg_label,
            text=alg_df["topology"],
            hovertemplate="%{text}<br>Utilización=%{y:.3f}<extra>%{fullData.name}</extra>",
        )
    )

fig_util.update_layout(
    title="Utilización de espectro por topología (orden TOP)",
    xaxis=dict(
        title="Índice TOP",
        tickmode="array",
        tickvals=tick_vals,
        ticktext=tick_text,
        rangeslider=dict(visible=True),
    ),
    yaxis=dict(title="Utilización"),
    legend_title_text="Algoritmo",
)
fig_util.show()


In [21]:
fig_frag = go.Figure()
for alg_label in df_metrics["algorithm"].unique():
    alg_df = df_metrics[df_metrics["algorithm"] == alg_label].sort_values("rank")
    fig_frag.add_trace(
        go.Scatter(
            x=alg_df["rank"],
            y=alg_df["fragmentation"],
            mode="lines+markers",
            name=alg_label,
            text=alg_df["topology"],
            hovertemplate="%{text}<br>Fragmentación=%{y:.2f}<extra>%{fullData.name}</extra>",
        )
    )

fig_frag.update_layout(
    title="Fragmentación promedio de espectro (segmentos libres)",
    xaxis=dict(
        title="Índice TOP",
        tickmode="array",
        tickvals=tick_vals,
        ticktext=tick_text,
        rangeslider=dict(visible=True),
    ),
    yaxis=dict(title="Fragmentación"),
    legend_title_text="Algoritmo",
)
fig_frag.show()


# Conexiones sin asignar

In [22]:
fig_blocked = go.Figure()
for alg_label in df_metrics["algorithm"].unique():
    alg_df = df_metrics[df_metrics["algorithm"] == alg_label].sort_values("rank")
    internal_key = LABEL_TO_KEY[alg_label]
    blocked_values = [all_results[row["topology"]][internal_key]["blocked_connections"] for _, row in alg_df.iterrows()]
    fig_blocked.add_trace(
        go.Scatter(
            x=alg_df["rank"],
            y=blocked_values,
            mode="lines+markers",
            name=alg_label,
            text=alg_df["topology"],
            hovertemplate="%{text}<br>Bloqueos=%{y}<extra>%{fullData.name}</extra>",
        )
    )

fig_blocked.update_layout(
    title="Conexiones sin asignar por topología",
    xaxis=dict(
        title="Índice TOP",
        tickmode="array",
        tickvals=tick_vals,
        ticktext=tick_text,
        rangeslider=dict(visible=True),
    ),
    yaxis=dict(title="Conexiones bloqueadas"),
    legend_title_text="Algoritmo",
)
fig_blocked.show()


### Resumen determinista de resultados

In [23]:
print("RESUMEN DETERMINISTA DE RMSA")

ordered_names = [payload["name"] for payload in sorted(loaded_topos.values(), key=lambda topo: topo["numeric_id"])]
for topo_name in ordered_names:
    topo_results = all_results.get(topo_name, {})
    if not topo_results:
        continue
    print(f"\nTopología: {topo_name}")
    print("-" * 50)
    metrics_by_alg = {alg: topo_results[alg] for alg in ALGORITHM_LABELS if alg in topo_results}
    if not metrics_by_alg:
        print("  Sin métricas disponibles.")
        continue
    best_util = max(metrics_by_alg.items(), key=lambda item: item[1]["utilization"])
    best_frag = min(metrics_by_alg.items(), key=lambda item: item[1]["fragmentation"])
    least_blocked = min(metrics_by_alg.items(), key=lambda item: item[1]["blocked_connections"])
    print(f"  Mayor utilización: {ALGORITHM_LABELS[best_util[0]]} ({best_util[1]['utilization']:.4f})")
    print(f"  Menor fragmentación: {ALGORITHM_LABELS[best_frag[0]]} ({best_frag[1]['fragmentation']:.2f})")
    print(f"  Menos bloqueos: {ALGORITHM_LABELS[least_blocked[0]]} ({least_blocked[1]['blocked_connections']})")



RESUMEN DETERMINISTA DE RMSA

Topología: TOP_1_ABILENE
--------------------------------------------------
  Mayor utilización: First-Fit (0.1214)
  Menor fragmentación: First-Fit (3.68)
  Menos bloqueos: First-Fit (0)

Topología: TOP_2_ANS
--------------------------------------------------
  Mayor utilización: First-Fit (0.2285)
  Menor fragmentación: First-Fit (6.78)
  Menos bloqueos: First-Fit (0)

Topología: TOP_3_ARNES
--------------------------------------------------
  Mayor utilización: First-Fit (0.2656)
  Menor fragmentación: First-Fit (7.33)
  Menos bloqueos: First-Fit (0)

Topología: TOP_4_ARPANET
--------------------------------------------------
  Mayor utilización: First-Fit (0.2125)
  Menor fragmentación: First-Fit (6.64)
  Menos bloqueos: First-Fit (0)

Topología: TOP_5_ATMNET
--------------------------------------------------
  Mayor utilización: First-Fit (0.5307)
  Menor fragmentación: First-Fit (14.43)
  Menos bloqueos: First-Fit (0)

Topología: TOP_6_BBNPLANET
----

### Exportar resultados agregados a CSV y JSON

In [ ]:
results_rows = []
for topo_name, topo_results in all_results.items():
    payload = loaded_topos[topo_name]
    workload_size = topo_results.get("workload_size", payload["n_connections"])
    for alg_key, alg_label in ALGORITHM_LABELS.items():
        metrics = topo_results.get(alg_key)
        if not metrics:
            continue
        results_rows.append({
            "topology": topo_name,
            "alias": payload["alias"],
            "topology_rank": payload["numeric_id"],
            "slots_per_link": payload["slots_per_link"],
            "algorithm": alg_label,
            "workload_size": workload_size,
            **metrics,
        })

df_results = pd.DataFrame(results_rows)
RESULTS_DIR.mkdir(exist_ok=True)
csv_path = RESULTS_DIR / "rmsa_results.csv"
df_results.to_csv(csv_path, index=False)
print(f"Resultados exportados a {csv_path}")

json_path = RESULTS_DIR / "rmsa_results.json"
with open(json_path, "w", encoding="utf-8") as handler:
    json.dump(all_results, handler, indent=2)
print(f"Resultados exportados a {json_path}")
